## Search Engine With Tools and Agents

In [5]:
## Arxiv--Research
## Tools Creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [7]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [9]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [10]:
tools = [wiki, arxiv]

In [14]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [16]:
loader = WebBaseLoader("https://docs.smith.langchain.com/docs")
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(document, HuggingFaceEmbeddings())
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCA427F400>, search_kwargs={})

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"Langchain-search","Search any information about langsmith")

retriever_tool.name

'Langchain-search'

In [18]:
tools=[wiki, arxiv, retriever_tool]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Vishal\\Multi-Model-Q-A-Chatbot-using-Open-AI\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langchain-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FCA5C3E170>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object a

In [22]:
## Run all this tools with Agents and LLM Models

## Tools, LLM--> AgentsExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os


groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")


In [23]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [25]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [26]:
agent_executor.invoke({"input": "What is Langchain?"})



> Entering new AgentExecutor chain...

Invoking: `Langchain-search` with `{'query': 'What is Langchain?'}`


🦜️🛠️ LangSmith








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.
Invoking: `wikipedia` with `{'query': 'LangSmith'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languagLangChain is a software framework that helps facilitate the integration of large language models (LLMs)

{'input': 'What is Langchain?',
 'output': "LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models themselves, such as natural language processing (NLP), machine translation, text generation, and question answering. LangChain's architecture is designed to be flexible and extensible, allowing developers to easily integrate LLMs into their applications."}

In [27]:
agent_executor.invoke({"input": "What is Machine Learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt seems like the tool call yielded a result that is a good starting point to answer your question. Here's a more detailed answer based on the result:

Machine learning is a subfield of artificial intelligence that involves the use of statistical models and algorithms to enable machines to learn from data and make decisions or predictions without being explicitly programmed. The goal of machine learning is to enable machines to generalize their learning to new, unseen data, and make accurate predictions or decisions.

In other words, machine learning is a type of AI that allows systems to automatically improve their performance on a task by

{'input': 'What is Machine Learning?',
 'output': "It seems like the tool call yielded a result that is a good starting point to answer your question. Here's a more detailed answer based on the result:\n\nMachine learning is a subfield of artificial intelligence that involves the use of statistical models and algorithms to enable machines to learn from data and make decisions or predictions without being explicitly programmed. The goal of machine learning is to enable machines to generalize their learning to new, unseen data, and make accurate predictions or decisions.\n\nIn other words, machine learning is a type of AI that allows systems to automatically improve their performance on a task by learning from experience and data, without being explicitly programmed. This is done by using algorithms that can recognize patterns in data, make predictions, and adapt to new information.\n\nMachine learning has many applications in various fields, including natural language processing, comput

In [34]:
agent_executor.invoke({"input": "What's 2503.22687 research paper about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2503.22687'}`


Published: 2025-03-05
Title: Qieemo: Speech Is All You Need in the Emotion Recognition in Conversations
Authors: Jinming Chen, Jingyi Fang, Yuanzhong Zheng, Yaoxuan Wang, Haojun Fei
Summary: Emotion recognition plays a pivotal role in intelligent hum
Invoking: `wikipedia` with `{'query': 'emotion recognition in conversations'}`


Page: Emotion recognition in conversation
Summary: Emotion recognition in conversation (ERC) is a sub-field of emotion recognition, that focuses on mining human emotions from conversations or dialogues having two or more interlocutors. The datasets iBased on the results of the previous tool calls, it seems that the research paper "Qieemo: Speech Is All You Need in the Emotion Recognition in Conversations" by Jinming Chen et al. (2025) is related to emotion recognition in conversations. The paper discusses a new method called Qieemo for emotion recognition in conversatio

{'input': "What's 2503.22687 research paper about?",
 'output': 'Based on the results of the previous tool calls, it seems that the research paper "Qieemo: Speech Is All You Need in the Emotion Recognition in Conversations" by Jinming Chen et al. (2025) is related to emotion recognition in conversations. The paper discusses a new method called Qieemo for emotion recognition in conversations using only speech data.\n\nAccording to the Wikipedia article on emotion recognition in conversation, this sub-field of emotion recognition focuses on mining human emotions from conversations or dialogues having two or more interlocutors. The datasets used for emotion recognition in conversation typically include transcribed conversations, audio recordings, or text-based chat logs.\n\nGiven this information, I can provide a direct answer to your original question:\n\nThe research paper "Qieemo: Speech Is All You Need in the Emotion Recognition in Conversations" by Jinming Chen et al. (2025) is about